# Exploiting ECDSA Nonce Biases

It's well known that leaking even a single nonce used with an ECDSA key leads to a key compromise. But what if we have some knowledge about the structure of a nonce `k`? For example, what if some of the high bits were set to zero? Since `k` is now much smaller the generator point's order `q`, we can use a system of linear equations to approximate `k`. When we have enough equations, finding the shortest vector a la LLL will recover the nonces used for each signature. This is an example of a reduction to the Hidden Number Problem.

Here is how the system of equations will be formatted:
```
k1 - d*r1/s1 - h1/s1 = 0 (mod q)
k2 - d*r2/s2 - h2/s2 = 0 (mod q)
...
kn - d*rn/sn - hn/sn = 0 (mod q)
```

That's the most basic formulation of the attack. But what if the high bits are fixed to a non-zero constant? Our attack relies on the fact that `k` is much smaller than `q`, so we need a way to zero out those bits. The solution is to simply use one signature as the "relative point", rewrite all other signatures in terms of it (i.e. just subtract), and then we get the difference of each nonce with the nonce of the relative term.

Let's draw this. `B` will stand for a biased bit that's fixed to a constant. `$` will stand for a random bit. We'll save the last equation `n` as our relative term.

```
  k1 BBBBBBBB$$$$$$$$
- kn BBBBBBBB$$$$$$$$
-------------------------
     00000000$$$$$$$$
```

Once we find a nonce differential, we can extract the private key by rewriting the equation for `x` using `k1`-`kn`.

`x = (sn*m1 – s1*mn – s1*sn(k1 – kn))/(rn*s1 – r1*sn) (mod q)`

Alright, but what if the bias is at the low bits of the nonces? We just right shift all of the equations' terms until they align with the LSB edge.

`$$$$$$$$00000000` >> 8 == `00000000$$$$$$$$`

When we recompute the nonce differential, we simply shift it back. Well, what if we don't have a bias, but we know the high bits of each nonce? Let's rewrite `k` as `a+b` where `a` is the known bits and `b` is the unknown bits.

```
d*r       = s*k - h     (mod q)
d*r       = s*(a+b) - h (mod q)
d*r/s     = a+b - h/s   (mod q)
d*r/s - b = a - h/s     (mod q)
```

Once we recover the nonce differentials, we just add in the difference between each nonce's `a` and `n`'s `a`. If we know the low bits, we do the same thing except right shift the values to realign them with the LSB edge. It's also possible there's a bias _and_ a partial leak. However, this is already solvable using our above tools as long as we make sure we're aligned correctly.